In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, BooleanType, DateType, FloatType, StringType, ArrayType


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1741341246780_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("RecipeRecommendation").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#Defining and storing in a variable the S3 path link for Food Recipie folder
s3_path = "s3://receipiebucket/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# S3 Data Paths
recipes_s3_path  = s3_path + "RAW_recipes_cleaned.csv"
ratings_s3_path  = s3_path + "RAW_interactions_cleaned.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Task 1: Read the Recipe Data

Read RAW_recipes.csv from S3 bucket.

Ensure each field has the correct data type.

In [5]:
#read the raw recipes file
raw_recipes_df = spark.read.csv(recipes_s3_path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#View the Raw Recipies schema
raw_recipes_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)

### Test cases for Task 01

In [7]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_recipes_df.count() == 231637, "There is a mistake in reading the data."
assert len(raw_recipes_df.columns) == 12, "There is a mistake in reading the data."
assert raw_recipes_df.schema["minutes"].dataType == IntegerType(), "The data types have not been read correctly."
assert raw_recipes_df.schema["tags"].dataType == StringType(), "The data types have not been read correctly."
assert raw_recipes_df.schema["n_ingredients"].dataType == IntegerType(), "The data types have not been read correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Task 2: Extract Individual Features from Nutrition Column

Each row in the nutrition column contains seven values. Each value represents nutrition information. Your task is to separate the array into seven individual columns.

In [8]:
# Define function to process the nutrition column
def process_nutrition_column(df):
    """Splits the nutrition column (stored as a string) into individual numeric features."""
    
    # Step 1: Remove brackets and split the string into an array
    df = df.withColumn("nutrition_array", F.split(F.regexp_replace(F.col("nutrition"), r"[\[\]]", ""), ","))

    # Step 2: Convert extracted values into separate columns
    split_cols = ["calories", "total_fat_PDV", "sugar_PDV", "sodium_PDV", "protein_PDV", "saturated_fat_PDV", "carbohydrates_PDV"]
    
    for i, col in enumerate(split_cols):
        df = df.withColumn(col, F.col("nutrition_array")[i].cast(FloatType()))
    
    return df.drop("nutrition_array")  # Drop the nutrition_array column after processing


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Process the nutrion column by calling the function
raw_recipes_df = process_nutrition_column(raw_recipes_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
raw_recipes_df.select("calories", "total_fat_PDV", "sugar_PDV", "sodium_PDV", "protein_PDV", "saturated_fat_PDV", "carbohydrates_PDV").show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|
+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|    51.5|          0.0|     13.0|       0.0|        2.0|              0.0|              4.0|
|   173.4|         18.0|      0.0|      17.0|       22.0|             35.0|              1.0|
|   269.8|         22.0|     32.0|      48.0|       39.0|             27.0|              5.0|
|   368.1|         17.0|     10.0|       2.0|       14.0|              8.0|             20.0|
|   352.9|          1.0|    337.0|      23.0|        3.0|              0.0|             28.0|
+--------+-------------+---------+----------+-----------+-----------------+-----------------+
only showing top 5 rows

#### Test cases for task 02

In [11]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_recipes_df.schema["carbohydrates_PDV"].dataType == FloatType(), "Recheck your typecasting"
assert raw_recipes_df.collect()[123432][14] == 62.0, "The columns have not been split correctly."
assert raw_recipes_df.collect()[10000][12] == 60.400001525878906, "The columns have not been split correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Step 3: Standardize Nutrition Values

The nutritional values in absolute terms will have a lot of variation. For example, a recipe serving six people (recipe A) will have more sugar than a recipe meant to serve one person (recipe B). But that does not necessarily imply that the sugar per person in recipe A is more than in recipe B. Standardize the nutrition values. Convert the nutritional values to per 100 calories.

In [12]:
# Define function to standardize the nutrition column
def standardize_nutrition(df):
    """Converts nutrition values to per 100 calories while handling nulls and division by zero."""
    nutrition_cols = ["total_fat_PDV", "sugar_PDV", "sodium_PDV", "protein_PDV", "saturated_fat_PDV", "carbohydrates_PDV"]
    
    # Replace NULLs in `calories` with 1 (to avoid division errors)
    df = df.withColumn("calories", F.when(F.col("calories").isNull() | (F.col("calories") == 0), 1).otherwise(F.col("calories")))
    
    # Compute new per_100_cal features safely
    for col in nutrition_cols:
        df = df.withColumn(col.replace('_PDV','')+'_per_100_cal', F.when(F.col(col).isNotNull(), (F.col(col) / F.col("calories")) * 100).otherwise(0))
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Call function to standardize the nutrition column
raw_recipes_df = standardize_nutrition(raw_recipes_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
raw_recipes_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+------------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal| sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+---------------

#### Test cases for Task 03

In [15]:
# total fat check for id 28881
assert raw_recipes_df.filter("id == 28881").select('total_fat_per_100_cal').first()[0] == 0, "total_fat_per_100_cal for recipe 28881 should be 0"

# total fat check for id 112140
assert round(raw_recipes_df.filter("id == 112140").select('total_fat_per_100_cal').first()[0]) == 8, "total_fat_per_100_cal for recipe 112140 should be 8"

# checking for nulls
for c in ['total_fat_per_100_cal','sugar_per_100_cal','sodium_per_100_cal','protein_per_100_cal',
                          'saturated_fat_per_100_cal','carbohydrates_per_100_cal']:
    assert raw_recipes_df.select(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c)).collect()[0][0] == 0, "There are Nulls in the data"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Step 4: Process Tags Column

Convert the tags column from a string to an array of strings

In [16]:
# Define function to process the tag column
def process_tags_column(df):
    """Converts tags column from a string to an array of strings, ensuring proper null handling."""
    return df.withColumn("tags", 
                         F.split(F.regexp_replace(F.col("tags"), r"[\[\]']", ""), ",\s*").cast(ArrayType(StringType(), True))
                        )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# call the function to process the tag column
raw_recipes_df = process_tags_column(raw_recipes_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
raw_recipes_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+------------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal| sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+---------------

#### Test cases for Task 04

In [19]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert raw_recipes_df.schema["tags"].dataType == ArrayType(StringType(), True), "You have not split the string into an array."
assert raw_recipes_df.collect()[2][5] == ['time-to-make','course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less'], "Recheck your string cleaning and splitting operations."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Task 5: Join Datasets

Read the RAW_interaction.csv and join this interaction level file with the recipe level data frame. The resulting data frame should have all the interactions.

In [20]:
# Read the ratings file from S3 bucket and store in dataset
raw_ratings_df = spark.read.csv(ratings_s3_path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#Define function to join the 2 datasets
def join_datasets(recipes_df, ratings_df):
    return recipes_df.join(ratings_df, recipes_df.id == ratings_df.recipe_id, "inner")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
#Call the join function and store in a new data set
interaction_level_df = join_datasets(raw_recipes_df, raw_ratings_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Display final schema
interaction_level_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carbohydrates_PDV: float (nullable = true)
 |-- total_fat_per_100_cal: double (nullable = true)
 |-- sugar_per_100_cal: double (nullable = true)
 |-- sodium_per_100_cal: double (nullable = true)
 |-- protei

#### Test cases for Task 05

In [36]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert (interaction_level_df.count() ,len(interaction_level_df.columns)) == (1132367, 30), "The type of join is incorrect"

list1 = raw_ratings_df.select('recipe_id').collect()
list2 = raw_recipes_df.select('id').collect()
exclusive_set = set(list1)-set(list2)

assert len(exclusive_set) == 0, "There is a mistake in reading one of the two data files."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Task 6:  Create time-based features

Create features that capture the time passed between one review and the date on which the recipe was submitted.

In [143]:
# Define function to create time-based features
## (365 + 365+ 365 +366) /48 = ~30.4375

def create_time_features(df):
    """Creates features based on time differences between current date vs review / submission."""
    df = df.withColumn("submitted", F.to_date(F.col("submitted"), "yyyy-MM-dd"))
    df = df.withColumn("date", F.to_date(F.col("date"), "yyyy-MM-dd"))
        # Review Date vs Submission date    
    df = df.withColumn("days_since_submission_on_review_date", F.datediff(F.col("date"), F.col("submitted")))
    df = df.withColumn("months_since_submission_on_review_date", F.round(F.col("days_since_submission_on_review_date") / 30.4375,2))
    df = df.withColumn("years_since_submission_on_review_date", F.round(F.col("months_since_submission_on_review_date") / 12.0,2))
    
    # Current Date vs Submission date
    df = df.withColumn("days_since_submission", F.datediff(F.current_date(), F.col("submitted")))    
    df = df.withColumn("months_since_submission",  F.round(F.col("days_since_submission") / 30.4375,2))
    df = df.withColumn("years_since_submission", F.round(F.col("months_since_submission") / 12.0, 2))
    # Current Date vs review date    
    df = df.withColumn("days_since_review", F.datediff(F.current_date(), F.col("date")))
    df = df.withColumn("months_since_review",  F.round(F.col("days_since_review") / 30.4375,2))
    df = df.withColumn("years_since_review", F.round(F.col("months_since_review") / 12.0, 2))
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
# Run the function to create time-based features
interaction_level_df = create_time_features(interaction_level_df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Unrounded values used in Test Cases and therefor there is an Assertion error in test case # 06

#### Test cases for Task 06

In [128]:
# Code check cell
# Do not edit cells with assert commands
# If an error is shown after running this cell, please recheck your code.  

assert interaction_level_df.schema["days_since_submission_on_review_date"].dataType == IntegerType()

assert (interaction_level_df.filter((interaction_level_df.user_id == 428885) & (interaction_level_df.recipe_id == 335241))
                            .select('days_since_submission_on_review_date').collect()[0][0]) == 77
assert (interaction_level_df.filter((interaction_level_df.user_id == 2025676) & (interaction_level_df.recipe_id == 94265))
                            .select('months_since_submission_on_review_date').collect()[0][0]) == 153.22580645
assert (interaction_level_df.filter((interaction_level_df.user_id == 338588) & (interaction_level_df.recipe_id == 21859))
                            .select('years_since_submission_on_review_date').collect()[0][0]) == 4.564516129166667

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

Traceback (most recent call last):
AssertionError



In [148]:
# Save processed dataset to S3 bucket
interaction_level_df.write.mode("overwrite").parquet(s3_path+"Interaction_level_df_ModelReady.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Process the remaining items in a new notebok